# Preprocessing for Analysis of Large-Scale Collection of Language-Level Provenance

Using the service [RaaS](https://www.github.com/jwons/raas), we executed thousands of R scripts hosted on Harvard's Dataverse while collecting langauge-level provenance using the `rdtLite` variant of the [RDataTracker](https://www.github.com/End-to-end-provenance/RDataTracker) collection tool. This notebook takes that raw data and processes it to generate an aggregate table of information about all the scripts to help find trends and patterns in R scripts or their provenance. 

## Description of Raw Data

The RaaS collection process resulted in ~100 GB of metadata we have stored in a directory ll-prov-data (not hosted on GitHub due to size). The directory structure is described below:
```
📦ll-prov-data
 ┣ 📂dataset_name1_prov_data
 ┃ ┗ 📂prov_data
 ┃ ┃ ┣ 📂dataset1_script_name1
 ┃ ┃ ┃ ┣ 📂data
 ┃ ┃ ┃ ┣ 📂debug
 ┃ ┃ ┃ ┣ 📂scripts
 ┃ ┃ ┃ ┃ ┗ 📜dataset1_script_name1.R
 ┃ ┃ ┃ ┗ 📜prov.json
 ┃ ┃ ┣ 📂dataset1_script_name2
 ┃ ┃ ┃ ┣ 📂data
 ┃ ┃ ┃ ┃ ┣ 📜recorded_text_data.txt
 ┃ ┃ ┃ ┃ ┣ 📜recorded_csv_data.csv
 ┃ ┃ ┃ ┣ 📂debug
 ┃ ┃ ┃ ┣ 📂scripts
 ┃ ┃ ┃ ┃ ┗ 📜dataset1_script_name2.R
 ┃ ┃ ┃ ┗ 📜prov.json
 ┃ ┃ ┣ 📜get_prov.RData
 ┃ ┃ ┗ 📜run_log.csv
 ┣ 📂dataset_name2_prov_data
 ┃ ┗ 📂prov_data
 ┃ ┃ ┣ 📂dataset2_script_name1
 ┃ ┃ ┃ ┣ 📂data
 ┃ ┃ ┃ ┣ 📂debug
 ┃ ┃ ┃ ┣ 📂scripts
 ┃ ┃ ┃ ┃ ┗ 📜dataset2_script_name1.R
 ┃ ┃ ┃ ┃ ┗ 📜sourced_script.R 
 ┃ ┃ ┃ ┗ 📜prov.json
 ┃ ┃ ┣ 📜get_prov.RData
 ┃ ┃ ┗ 📜run_log.csv
 ```
 Dataverse hosts 'datasets' which contain artifacts from published research, in this context that means one or more scripts. Our raw data is split into this granularity, with a directory for the provenance of each dataset (e.g. `dataset_name1_prov_data`). Within the dataset directory, each script has a directory (e.g. `dataset1_script_name1`). Within a script's directory is a `prov.json` file which contains the provenance graph, and three directories. The most important one for this dataset is the `scripts` directory which contains the script we collected provenance from, and any scripts the original one called (`source`d in R lingo). 
For this analysis, the run_log.csv files and get_prov.RData files in a dataset's directory can be ignored as they are artifacts from the RaaS evaluation.

## Preprocessing the Raw Data into a Table

We preprocess this raw data to generate a table that collects information on the number of each node and edge type in the provenance graphs, as well as features like number of lines in the script. We can use this table to then generate aggregate statistics. 

In [1]:
# Libraries to search the file system, parallelize, and parse provenance. 
import os
import ray

import pandas as pd

from glob import glob 
from provdebug import ProvParser

In [2]:
# We wrote this analysis to execute in a Docker container created from the jupyter/scipy-notebook:4d9c9bd9ced0 image. Therefore, we know this will be the path to the data. 
# For running this analysis outside of this environment, the path will need to change. 
root_directory = "/home/jovyan/work/"

# First we must walk through all the directories described above and find all the `prov.json` files
json_files = [y for x in os.walk(root_directory) for y in glob(os.path.join(x[0], 'prov.json'))]

In [3]:
# This function will take the full file path to the script from a provenance file, and produced a correctly formatted DOI of the dataset the script belonged to
def get_doi_from_script_name(dir_path):
    doi = dir_path.split("/home/rstudio/")[1]
    doi = doi.split("/dataset/")[0]
    doi = doi.replace("-", ":", 1)
    doi = doi.replace("-", "/")
    return(doi)

# this function returns the number of lines from a script
def get_num_of_lines(filename, change_encoding = False):
    
    # Some files aren't in the default encoding 
    if not change_encoding:
        script_file = open(filename, "r")
    else:
        script_file =  open(filename, "r", encoding = "latin1")
    lines = script_file.readlines()
    script_file.close()
    
    # Only count lines with code in them 
    num_of_lines = len([line for line in lines if line != "\n"])
    
    return(num_of_lines)
    
# This function will extract information about which external libraries and their functions are called in the R scripts
# It returns a dictionary where the keys represent column names and the values the column values. 
# When this function is called it will be on many filenames which will later be combined into one dataframe
@ray.remote
def get_function_edge_data(filename):
    # We wrote a provenance parser to allow easy access to the info stored in the json, it is intialized by passing the filepath to the prov.json
    try:
        prov_info = ProvParser.Parser(filename)
    except:
        return("Error in: " + filename)
    
    # This dataframe maps function node names to library node names 
    # It uses node names, not names of the libraries or functions themselves (f1 -> l1, f7 -> l3, etc)
    func_lib_df = prov_info.getFuncLib()
    
    # The eventual return value, declare it here as empty in case there are no functions and libraries
    # so that the function will still return just the empty dict with correct keys
    func_df = {"library_name":[], "function_name":[], "script_name":[], "doi":[]}
    
    # Check if there are even function library edges, if so collect that info
    if "collection" in func_lib_df:
        # Use the function labels from func_lib_df to index into the function node df and return the names of the functions the script called 
        func_names = prov_info.getFuncNodes().loc[func_lib_df["entity"]]["name"].values
        
        # Use the library labels from func_lib_df to index into the library node df and return the names of the libraries the script called functions from
        lib_names = prov_info.getLibs().loc[func_lib_df["collection"]]["name"].values
        
        # Script name and doi to uniquely identify where a function -> library mapping originated from 
        script_name = os.path.basename(prov_info.getEnvironment().loc["script"][0])
        doi = get_doi_from_script_name(prov_info.getEnvironment().loc["script"][0])
        
        # Combine into a dict to return and later be constructed into a dataframe
        func_df["library_name"] = lib_names
        func_df["function_name"] = func_names
        func_df["script_name"] = [script_name] * len(func_names)
        func_df["doi"] = [doi] * len(func_names)
        
    return(func_df)
    
# This function will take a provenance file and extract some summary information before returning a row for the aggregate table. 
@ray.remote
def get_prov_info_from_file(filename):
    
    # We wrote a provenance parser to allow easy access to the info stored in the json, it is intialized by passing the filepath to the prov.json
    try:
        prov_info = ProvParser.Parser(filename)
    except:
        return("Error in: " + filename)

    # One feature we identify is number of lines in the original script
    # Due to encoding errors, and the fact that the scripts directory can contain more than the original script,
    # we need multiple ways to find this file. We use the following variable to do this. 
    script_name = os.path.basename(prov_info.getEnvironment().loc["script"][0])
    script_path = os.path.dirname(filename) + "/scripts/" + script_name
    script_directory = os.path.dirname(filename) + "/scripts/"
    
    # If the script path exists, that's the original script and we can use it directly
    if(os.path.exists(script_path)):
        
            # Sometimes there are encoding errors, and if it doesn't work with the default, 
            # we've found that `latin1` will work correctly
            try:
                num_of_lines = get_num_of_lines(script_path)
            except UnicodeDecodeError:
                num_of_lines = get_num_of_lines(script_path, change_encoding=True)
                
    else:
        # The filename might not match correctly due to the way it was encoded in the provenance,
        # but in all cases of this there should only be one file in the scripts directory, use that one
        r_files = [y for x in os.walk(script_directory) for y in glob(os.path.join(x[0], '*.R'))]
        if(len(r_files) == 1):
            num_of_lines = get_num_of_lines(r_files[0])
        # This was used during debugging, it should NOT execute now. If it does, the data is different or something is wrong 
        else:
            print("Unsure about R files:" + filename)
            num_of_lines = 0
        
    # Create the row as a list. Later, we will pass the list of lists generate a DataFrame
    prov_values = [get_doi_from_script_name(prov_info.getEnvironment().loc["script"][0]), #doi
        os.path.basename(prov_info.getEnvironment().loc["script"][0]), #script_name
        num_of_lines, #num_of_lines
        False if prov_info.getDataNodes().empty else prov_info.getDataNodes().name.eq('error.msg').any(), #error
        len(prov_info.getProcNodes()), #num_of_proc_nodes 
        len(prov_info.getDataNodes()), #num_of_data_nodes 
        len(prov_info.getLibs()), #num_of_libraries 
        len(prov_info.getFuncNodes()), #num_of_functions 
        len(prov_info.getProcData()), #num_of_pd_edges 
        len(prov_info.getDataProc()), #num_of_dp_edges 
        len(prov_info.getFuncProc())] #num_of_fp_edges

    return(prov_values)      

In [4]:
# Intialize the multithreading library so we can use all CPUs so this doesn't take forever
ray.init(ignore_reinit_error=True)  

2022-11-03 17:52:47,369	INFO worker.py:1518 -- Started a local Ray instance.


Python version:,3.8.8
Ray version:,2.0.1


In [5]:
# prov_results becomes a list of lists where each inner list is a row of the table
# This will become the "Aggregate Table" described below
prov_results = ray.get([get_prov_info_from_file.remote(json_file) for json_file in json_files])

In [6]:
# func_results_dicts becomes a list of dicts where the key is the column names and the values are the elements of that column
# This will become the "Function Table" described below
func_results_dicts = ray.get([get_function_edge_data.remote(json_file) for json_file in json_files])
# Combine the dicts into a single dataframe
func_results = pd.concat([pd.DataFrame(func_dict) for func_dict in func_results_dicts])

In [7]:
# No more multithreading after this point
ray.shutdown()

## Aggregate Table

The following cell will create the final table containing information on the graph elements for each provenance file, as well as metadata features. We write the completed table out to `prov_table.csv` in the `output` directory. 

The following table describes each feature in the table. For more information on the provenance terms, see the [W3C PROV_JSON description](https://www.w3.org/Submission/prov-json/) and the RDataTracker [Extended PROV-JSON](https://github.com/End-to-end-provenance/ExtendedProvJson/blob/master/JSON-format.md)

| Feature | Description |
|--------:|:------------|
|doi      | Uniquely identifies each dataset, and can be used to find the original dataset on Dataverse|
|script_name | The name of the script we collected provenance for |
| num_of_lines | The number of lines in the original script |
| error   | Boolean value indicating whether the script encountered an error during execution (True) or not (False)|
| num_of_proc_nodes | The number of procedure nodes in the provenance graph |
| num_of_data_nodes | The number of data nodes in the provenance graph |
| num_of_libraries | The number of library nodes, i.e. the number of R packages loaded in the R environment while the script executed |
| num_of_functions | The number of function nodes, i.e. the number of unique functions called from external libraries |
| num_of_pd_edges | The number of procedure node to data node edges |
| num_of_dp_edges | The number of data node to procedure node edges |
| num_of_fp_edges | The number of function node to procedure edges |

The table is formatted as follows:

|    | doi                    | script_name                                                   |   num_of_lines | error   |   num_of_proc_nodes |   num_of_data_nodes |   num_of_libraries |   num_of_functions |   num_of_pd_edges |   num_of_dp_edges |   num_of_fp_edges |
|---:|:-----------------------|:--------------------------------------------------------------|---------------:|:--------|--------------------:|--------------------:|-------------------:|-------------------:|------------------:|------------------:|------------------:|
|  0 | doi:XX.XXXX/XXX/XXXXXX | script_name1.R                |            583 | True    |                  20 |                   9 |                 17 |                  2 |                 6 |                 8 |                 2 |
|  1 | doi:XX.XXXX/XXX/XXXXXX | script_name2.R                                 |            258 | False    |                   4 |                   2 |                  9 |                  0 |                 2 |                 1 |                 0 |



In [8]:
# Generate the table 
column_names= ["doi","script_name","num_of_lines", "error", "num_of_proc_nodes", "num_of_data_nodes", "num_of_libraries", "num_of_functions", "num_of_pd_edges", "num_of_dp_edges", "num_of_fp_edges"]
prov_df = pd.DataFrame(prov_results, columns = column_names)

# No procedure nodes means the execution likely failed
prov_df = prov_df[prov_df.num_of_proc_nodes != 0]

# No data nodes is highly likely to have failed, or a script that only loads libraries
prov_df = prov_df[prov_df.num_of_data_nodes != 0]

# Our completed table, re-index and save it and print descriptive stats
prov_df.index = range(0, len(prov_df))
prov_df.to_csv("../output/prov_table.csv")
prov_df.describe()

,num_of_lines,num_of_proc_nodes,num_of_data_nodes,num_of_libraries,num_of_functions,num_of_pd_edges,num_of_dp_edges,num_of_fp_edges
count,11463.000000,11463.000000,11463.000000,11463.000000,11463.000000,11463.000000,11463.000000,11463.000000
mean,337.420309,34.392218,29.968158,14.993457,5.075547,27.453110,37.263195,17.673820
std,777.592910,83.042793,111.256454,6.876395,8.038245,101.502168,123.474316,63.578517
min,1.000000,3.000000,1.000000,8.000000,0.000000,0.000000,0.000000,0.000000
25%,66.000000,6.000000,1.000000,9.000000,0.000000,1.000000,0.000000,0.000000
50%,143.000000,12.000000,5.000000,13.000000,1.000000,4.000000,2.000000,1.000000
75%,347.000000,31.000000,26.000000,19.000000,7.000000,22.500000,28.000000,13.000000
max,31931.000000,3612.000000,6825.000000,69.000000,83.000000,6824.000000,4626.000000,4155.000000


In [9]:
# Descriptive stats for only successful executions
successful_prov = prov_df[prov_df.error == False]
successful_prov.describe()

,num_of_lines,num_of_proc_nodes,num_of_data_nodes,num_of_libraries,num_of_functions,num_of_pd_edges,num_of_dp_edges,num_of_fp_edges
count,4069.000000,4069.000000,4069.000000,4069.000000,4069.000000,4069.000000,4069.000000,4069.000000
mean,256.472352,57.816171,53.882526,14.410174,9.009093,50.223888,72.139101,34.314819
std,697.267264,98.103454,134.264386,6.449837,9.434850,112.849417,169.106466,69.948031
min,1.000000,3.000000,1.000000,9.000000,0.000000,0.000000,0.000000,0.000000
25%,53.000000,12.000000,7.000000,9.000000,2.000000,6.000000,4.000000,2.000000
50%,119.000000,28.000000,23.000000,12.000000,6.000000,21.000000,25.000000,13.000000
75%,265.000000,65.000000,60.000000,18.000000,14.000000,56.000000,78.000000,37.000000
max,31931.000000,1930.000000,5279.000000,69.000000,74.000000,3530.000000,4626.000000,1536.000000


# Function Table

The following cell will create the final table containing information on which functions scripts call and the libraries they come from. We write the completed table out to `func_table.csv` in the `output` directory.

The following table describes each feature in the function table. 

| Feature | Description |
|--------:|:------------|
|doi      | Uniquely identifies each dataset, and can be used to find the original dataset on Dataverse|
|script_name | The name of the script we collected provenance for |
| function_name   | The name of the function the script identified in script_name called |
| library_name | The name of the library the function identified in function_name comes from |

The table is formatted as follows:

|    | doi                    | script_name                                                   |   function_name | library_name   |
|---:|:-----------------------|:--------------------------------------------------------------|----------------:|:---------------|
|  0 | doi:XX.XXXX/XXX/XXXXXX | script_name1.R                                                |            plot | graphics       |
|  1 | doi:XX.XXXX/XXX/XXXXXX | script_name2.R                                                | install.packages| utils          |

In [10]:
func_results.index = range(0, len(func_results))
func_results = func_results[["doi", "script_name", "function_name", "library_name"]]

func_results.to_csv("../output/func_table.csv")

func_results.describe()

,doi,script_name,function_name,library_name
count,57887,57887,57887,57887
unique,2830,6837,2609,670
top,doi:10.7910/DVN/VTUSLV,analysis.R,read.csv,ggplot2
freq,284,599,2524,17561
